In [1]:
import pandas as pd
import re


In [2]:
df = pd.read_csv('data.csv')
variables = pd.read_csv('../../../variables.csv')

## Merge data with the variables file

In [3]:
df['Model_simplified'] = df['Model - new table'].apply(lambda x: re.findall('\w+', x))
df['Model_simplified'] = df['Model_simplified'].apply(lambda x: x[0])
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\gabriel.genelot\AppData\Local\Temp\ipykernel_27232\2487617104.py:1: SyntaxWarning: invalid escape sequence '\w'
  df['Model_simplified'] = df['Model - new table'].apply(lambda x: re.findall('\w+', x))


,Name,Equation number,Model - new table,Input variable,Output variables,Equation shape,Formule,Zotero reference,IPCC risks,Input categories,Output categories,Latex fragment,Vensim,Vensim name,Model_simplified
0,Total agriculture impact,A.1,FUND v.3.8,"Damage in agricultural production,Damage in ag...",Total agriculture impact,NaN,a1.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food production reduction,Food security reduction",Production,Production,"A_{t,r}=A_{t,r}^{r}+A_{t,r}^{l}+A_{t,r}^{f}",Oui,FUND total agricultural impact,FUND
1,Agriculture impact of the rate of climate change,A.2,FUND v.3.8,"Time,Region,Change in regional mean temperatur...",Damage in agricultural production,NaN,a2.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food security reduction,Food production reduction",", Temperature",Production,"A_{t,r}^{r}=\alpha_{r}\left(\frac{\Delta T_{t}...",Oui,NaN,FUND
2,Agriculture impact of the level of climate change,A.3,FUND v.3.8,"Time,Region,Change in regional mean temperatur...",Damage in agricultural production level,NaN,a3.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food production reduction,Food security reduction",", Temperature",Production,"A_{t,r}^{l}\,=\,\delta_{r}^{l}T_{t}\,+\,\delta...",Oui,"""FUND - agricultural impact of the level of cl...",FUND
3,Agriculture impact of fertilisation,A.4,FUND v.3.8,"Time,Region,Pre-industrial concentration of CO...",Damage in agricultural production due to CO2 f...,NaN,a4.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food security reduction,Food production reduction",", CO2 concentration",Production,"A_{t,r}^{f}=\nu_{r}\ln\frac{G O2_{t}}{275}\,,",Oui,"""FUND - agricultural impact of the fertilisation""",FUND
4,Agriculture production relative loss,A.5,FUND v.3.8,"Gross agricultural product,GDP in region r at ...",NaN,NaN,a5.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food production reduction,Food security reduction","Production, GDP,",NaN,"\frac{G A P_{t,r}}{Y_{t,r}}=\frac{G A P_{1990,...",Oui,NaN,FUND


In [6]:
variables['Model'] = variables['Model'].str.strip()
variables['Equation'] = variables['Equation'].str.strip()
df['Model_simplified'] = df['Model_simplified'].str.strip()
df['Equation number'] = df['Equation number'].str.strip()



merged = variables.merge(df, right_on=['Model_simplified', 'Equation number'], left_on=['Model', 'Equation'])
merged.columns

Index(['Unnamed: 0', 'Real Name', 'Py Name', 'Subscripts', 'Units', 'Limits',
       'Type', 'Subtype', 'Comment', 'Model', 'Equation', 'isEquation', 'Name',
       'Equation number', 'Model - new table', 'Input variable',
       'Output variables', 'Equation shape', 'Formule', 'Zotero reference',
       'IPCC risks', 'Input categories', 'Output categories', 'Latex fragment',
       'Vensim', 'Vensim name', 'Model_simplified'],
      dtype='object')

In [11]:
merged['Input variable'] = merged['Input variable'].str.split(',')
merged['Output variables'] = merged['Output variables'].str.split(',')
merged.columns

Index(['Unnamed: 0', 'Real Name', 'Py Name', 'Subscripts', 'Units', 'Limits',
       'Type', 'Subtype', 'Comment', 'Model', 'Equation', 'isEquation', 'Name',
       'Equation number', 'Model - new table', 'Input variable',
       'Output variables', 'Equation shape', 'Formule', 'Zotero reference',
       'IPCC risks', 'Input categories', 'Output categories', 'Latex fragment',
       'Vensim', 'Vensim name', 'Model_simplified'],
      dtype='object')

In [20]:
import os

rst_file ='Damage functions\n===============\n\n'

# Filter the dataset to only equations
df = merged[merged["isEquation"]==True]


# Store the model name in a variable
current_model = None


for row in df.iterrows():

    if current_model != row[1]['Model - new table']:
        rst_file += row[1]['Model - new table'] + '\n' + '-'*len(row[1]['Model - new table']) + '\n\n'
    else:
        pass

    current_model = row[1]['Model - new table']

    rst_file += str(row[1]['Equation number']) + " : "  + row[1]['Name'] + '\n' + '~'*(3+len(row[1]['Name'])+len(str(row[1]['Equation number']))) + '\n\n'

    try: 
        rst_file += '.. math:: \n\n'
        rst_file += '   '+ row[1]['Latex fragment'] + ' \n\n'
    except:
        pass

    rst_file += 'Input variables:\n\n'

    try:
        for input in row[1]['Input variable']:
            rst_file += '- ' + input + '\n'
    except:
        pass

    rst_file += '\nOutput variables:\n\n'
    
    try: 
        for output in row[1]['Output variables']:
            rst_file += '- ' + output + '\n'
        rst_file += '\n\n'
    except:
        pass

    rst_file += '.. autofunction:: WILIAM.WILIAM.' + row[1]['Py Name'] + ' \n\n'


    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    file_path = os.path.join(parent_dir, '4_API/output.rst')


    with open(file_path, 'w') as file:
        file.write(rst_file)

In [19]:
merged

,Unnamed: 0,Real Name,Py Name,Subscripts,Units,Limits,Type,Subtype,Comment,Model,...,Equation shape,Formule,Zotero reference,IPCC risks,Input categories,Output categories,Latex fragment,Vensim,Vensim name,Model_simplified
0,8,DICE: 5: EQ damage function,dice_5_eq_damage_function,NaN,NaN,"(nan, nan)",Auxiliary,Normal,NaN,DICE,...,NaN,5.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l l}{{\ }}&{{\displaystyle=\psi_...,Oui,NaN,DICE
1,9,DICE: 5: damage function phi 1,dice_5_damage_function_phi_1,NaN,NaN,"(nan, nan)",Constant,Normal,NaN,DICE,...,NaN,5.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l l}{{\ }}&{{\displaystyle=\psi_...,Oui,NaN,DICE
2,10,DICE: 5: damage function phi2,dice_5_damage_function_phi2,NaN,NaN,"(nan, nan)",Constant,Normal,NaN,DICE,...,NaN,5.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l l}{{\ }}&{{\displaystyle=\psi_...,Oui,NaN,DICE
3,11,DICE: 6: EQ abatment function,dice_6_eq_abatment_function,NaN,NaN,"(nan, nan)",Auxiliary,Normal,NaN,DICE,...,NaN,6.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l}{{\Lambda({\bf t})\;=\;\theta_...,Oui,NaN,DICE
4,12,DICE: 6: Emissions control rate,dice_6_emissions_control_rate,NaN,NaN,"(nan, nan)",Constant,Normal,TODO: checker cette valeur là,DICE,...,NaN,6.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l}{{\Lambda({\bf t})\;=\;\theta_...,Oui,NaN,DICE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,148,FUND: W.1: alpha parameter,fund_w1_alpha_parameter,['REGIONS 35 I'],NaN,"(nan, nan)",Constant,External,α is a parameter (in percent of 1990 GDP per d...,FUND,...,NaN,w1.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Loss in freshwater ecosystem,Water security re...",", GDP, Population, Temperature",NaN,"W_{t,r}=\operatorname*{min}\left\{\alpha_{r}Y_...",Oui,NaN,FUND
151,149,FUND: W.1: beta parameter,fund_w1_beta_parameter,NaN,NaN,"(nan, nan)",Constant,Normal,"β = 0.85 (0.15, &gt;0) is a parameter, that sp...",FUND,...,NaN,w1.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Loss in freshwater ecosystem,Water security re...",", GDP, Population, Temperature",NaN,"W_{t,r}=\operatorname*{min}\left\{\alpha_{r}Y_...",Oui,NaN,FUND
152,150,FUND: W.1: gamma parameter,fund_w1_gamma_parameter,NaN,NaN,"(nan, nan)",Constant,Normal,"γ = 1 (0.5,&gt;0) is a parameter, that determi...",FUND,...,NaN,w1.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Loss in freshwater ecosystem,Water security re...",", GDP, Population, Temperature",NaN,"W_{t,r}=\operatorname*{min}\left\{\alpha_{r}Y_...",Oui,NaN,FUND
153,151,FUND: W.1: mu parameter,fund_w1_mu_parameter,NaN,NaN,"(nan, nan)",Constant,Normal,"η = 0.85 (0.15,&gt;0) is a parameter that spec...",FUND,...,NaN,w1.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Loss in freshwater ecosystem,Water security re...",", GDP, Population, Temperature",NaN,"W_{t,r}=\operatorname*{min}\left\{\alpha_{r}Y_...",Oui,NaN,FUND


## Trying to write documentation in the mdl file to have the correct form

In [18]:
# Example documentation 

def read_vensim(mdl_file, data_files=None, initialize=True,
                missing_values="warning", split_views=False,
                encoding=None, **kwargs):
    """
    Construct a model from Vensim `.mdl` file.

    Parameters
    ----------
    mdl_file: str or pathlib.Path
        The relative path filename for a raw Vensim `.mdl` file.

    initialize: bool (optional)
        If False, the model will not be initialize when it is loaded.
        Default is True.

    data_files: list or str or None (optional)
        If given the list of files where the necessary data to run the model
        is given. Default is None.

    missing_values: str ("warning", "error", "ignore", "keep") (optional)
        What to do with missing values. If "warning" (default)
        shows a warning message and interpolates the values.
        If "raise" raises an error. If "ignore" interpolates
        the values without showing anything. If "keep" it will keep
        the missing values, this option may cause the integration to
        fail, but it may be used to check the quality of the data.

    split_views: bool (optional)
        If True, the sketch is parsed to detect model elements in each
        model view, and then translate each view in a separate Python
        file. Setting this argument to True is recommended for large
        models split in many different views. Default is False.

    encoding: str or None (optional)
        Encoding of the source model file. If None, the encoding will be
        read from the model, if the encoding is not defined in the model
        file it will be set to 'UTF-8'. Default is None.

    subview_sep: list
        Characters used to separate views and subviews (e.g. [",", "."]).
        If provided, and split_views=True, each submodule will be placed
        inside the directory of the parent view.

    **kwargs: (optional)
        Additional keyword arguments for translation.

    Returns
    -------
    model: a PySD class object
        Elements from the Python model are loaded into the PySD class
        and ready to run

    Examples
    --------
    >>> model = read_vensim('../tests/test-models/samples/teacup/teacup.mdl')

    """

    return True

In [ ]:
for eq in df.itterows():

    output = ''

    # Adding description
    output += eq[1]['Description'] + '\n\n'

    # Adding input parameters
    output += 'Parameters\n----------\n'

    for param in eq[1]['Input variables']: 
        output += param['Py Name'] + ': \n'
        output += '    ' + param['Description'] + '\n\n'

    # Adding return = output variable

    for output in eq[1]['Output variables']:
        output += param['Py Name'] + ': \n'
        output += '    ' + param['Description'] + '\n\n'

print(output)

## Trying to add many-to-many relations in the nocodb

In [46]:
import pandas as pd
import requests

# Configuration
api_url = 'https://app.nocodb.com'
project_id = 'pgxzs6196j7d22v'
table_a_id = 'mjlzlvkuysvdvuq' # functions
table_b_id = 'm0gqe4gfebcvib3' # models
api_key = 'b0GndcsjgOx20Rnva1_3Rjvp5erw9fw6u-dALzqC'

# Headers pour l'API
headers = {
    'xc-auth': api_key,
    'Content-Type': 'application/json'
}

# Charger le fichier CSV
#csv_file_path = 'path_to_your_file.csv'
data = merged.copy()

# Fonction pour ajouter une entrée à une table
def add_entry(table_id, entry):
    response = requests.post(f'{api_url}/api/v1/{project_id}/{table_id}', headers=headers, json=entry)
    return response.json()

# Fonction pour obtenir ou créer une entrée dans TableB
def get_or_create_table_b_entry(value):
    # Vérifiez si l'entrée existe déjà
    response = requests.get(f'{api_url}/api/v2/{project_id}/{table_b_id}', headers=headers, params={'where': f'Model="{value}"'})
    result = response.json()
    if result:
        return result[0]['Id']
    else:
        # Sinon, créez la nouvelle entrée
        entry = {'Model': value}
        response = add_entry(table_b_id, entry)
        return response['Id']

# Traiter les données et ajouter des entrées dans TableA avec des relations many-to-many
for index, row in data.iterrows():
    # Créez ou récupérez les relations many-to-many
    related_ids = [get_or_create_table_b_entry(value) for value in row['Model - new table'].split(',')]
    
    # Ajouter l'entrée dans TableA
    table_a_entry = {
        'Field1': row['Field1'],
        'Field2': row['Field2'],
        # Ajoutez d'autres champs ici
        'relationField': related_ids  # Remplacez relationField par le nom réel du champ de relation many-to-many
    }
    
    add_entry(table_a_id, table_a_entry)

print("Données importées avec succès dans NocoDB.")


KeyError: 0

In [2]:
import requests

access_token = "b0GndcsjgOx20Rnva1_3Rjvp5erw9fw6u-dALzqC"

# Configuration
api_url = 'https://app.nocodb.com'
project_id = 'pgxzs6196j7d22v'
table_a_id = 'mjlzlvkuysvdvuq' # functions
table_b_id = 'm0gqe4gfebcvib3' # models

s = requests.Session()

# Create row
create_row_request = requests.post(f'https://app.nocodb.com/api/v1/{project_id}/{table_a_id}',
                  headers={'Content-Type': 'application/json',
                           'xc-token': access_token,
                           },
                  json={
                      "Model": "titleB"
                  }
                  )

# Find row
find_one_request = requests.get(f'https://app.nocodb.com/api/v1/{project_id}/{table_b_id}/find-one?where=(Model,eq,titleB)',
                                headers={'Content-Type': 'application/json',
                                         'xc-token': access_token,
                                         }
                                )
if find_one_request.status_code == 200:
    # Proceed to extract 'Id'
    row_id = find_one_request.json().get("Id")
    if row_id is None:
        print("Id not found in response.")
else:
    print(f"Request failed with status code: {find_one_request.status_code}")

row_id = find_one_request.json()["Id"]

# Get row id (other table)
find_one_request2 = requests.get(f'https://app.nocodb.com/api/v1/{project_id}/{table_a_id}/find-one?where=(Model,eq,titleA)',
                                 headers={'Content-Type': 'application/json',
                                          'xc-token': access_token,
                                          }
                                 )
row_id_other_table = find_one_request2.json()["Id"]


# Link row
link_row = requests.post(f'https://app.nocodb.com/api/v1/{project_id}/{table_b_id}/{row_id}/mm/columnName/{row_id_other_table}',
                         headers={'Content-Type': 'application/json',
                                  'xc-token': access_token,
                                  }
                         )
print(f"Status Code: {link_row.status_code}, Response: {link_row.json()}")

Request failed with status code: 404


KeyError: 'Id'